In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import naludaq
print(f"Naludaq version: {naludaq.__version__}")

FTDI import failed: FTDI driver not installed
D3XX import failed: D3XX driver not installed


Naludaq version: 0.31.7


In [3]:
from naludaq.backend import DiskAcquisition

ACQ_PATH = "data/SineWave_100MHz/"
channels = 4
# DiskAcquisition is recommended to be used as a context manager for safety
acq= DiskAcquisition(ACQ_PATH)
print(acq[0].keys())
start = np.zeros((1013))
print("start window:")
print(acq[76]['headers'])    #%  128       

# print(start.max())
# plt.hist(start, bins=30);
# plt.xlabel("start_window")
# plt.ylabel("counts")

dict_keys(['window_labels', 'data', 'start_window', 'window_timings', 'headers', 'time', 'created_at', 'pkg_num', 'event_num', 'name'])
start window:
{'channels': [15, 15], 'num_winds': [1, 1], 'event_id': [75, 75], 'prev_final_window': [3919, 3660], 'trigger_time_ns': [1929690, 7786092]}


In [4]:
with DiskAcquisition(ACQ_PATH) as acq:
    print("Board model:", acq.params)
    print("Number of events:", len(acq))
    print("Pedestals stored:", acq.pedestals is not None)


Board model: {'model': 'dsa-c10-8', 'channels': 8, 'windows': 510, 'samples': 64, 'chanmask': 61440, 'chanshift': 10, 'chips': {'0': {'channels': 4, 'name': 'aardvarcv3', 'readable_name': 'AARDVARCv3', 'resolution': 12, 'samples': 64, 'windows': 510}, '1': {'channels': 4, 'name': 'aardvarcv3', 'readable_name': 'AARDVARCv3', 'resolution': 12, 'samples': 64, 'windows': 510}}, 'clock_file': '/root/nalu/_internal/naluconfigs/data/clocks/Si5341_TRBHM_10GSaps_GCCsstx2_Registers.txt', 'connections': ['serial', 'd2xx', 'udp'], 'default_baud': {'115200': 868}, 'default_baudrate': 115200, 'default_clock': 100000000.0, 'default_divider': 868, 'default_trigger_value': 1500, 'ext_dac': {'channel_mapping': {'0': 0, '1': 2, '2': 5, '3': 7, '4': 0, '5': 2, '6': 5, '7': 7, 'cal0': 4, 'cal1': 4}, 'channels': {'0': 1702, '1': 1702, '2': 1702, '3': 1702, '4': 2218, '5': 1702, '6': 1702, '7': 1702}, 'chip': 'ltc2620', 'chip_mapping': {'0': 0, '1': 0, '2': 0, '3': 0, '4': 1, '5': 1, '6': 1, '7': 1, 'cal0': 